# A Simple Deterministic Example with a Mortality Crisis

In [5]:
import os
os.chdir('..')
from households import *
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd

In [6]:
from households import *

In [14]:
class MortalityCrisis(object):
    """ A simulation-wide population crisis event with customizable start year, duration, and removal probabilities.
    
    When called, takes world as an argument and iterates through all community and household objects, 
    deciding for each person object whether or not they will be removed from the simulation whiling recording number of 
    removed and changes in household structure across each time step of the event.
    
    Parameters
    ----------
    startyear: int
        defines starting year of crisis
    duration: int
        defines lenth and subsequent ending year of crisis
    plaguetab: list
        list of agetables for determining probability of removal based on household size
    """
    def __init__(self, startyear, endyear, plaguetab):
        self.startyear = startyear
        self.duration = duration
        if isinstance(plaguetab, AgeTable) == False:
           raise TypeError('plaguetab not of type AgeTable')
        self.plaguetab = plaguetab
    
    def __call__(self, world):
        if terra.year < plague.startyear or terra.year > plague.startyear + plague.duration:
            pass
        else:
            for community in world.communities:
                for house in community.houses:
                    for person in house.people:
                        if len(house.people) <= 5:
                            r = self.plaguetab[0].get_rate(person.sex, person.age)
                        if 5 < len(house.people) <=10:
                            r = self.plaguetab[1].get_rate(person.sex, person.age)
                        if len(house.people) > 10:
                             r = self.plaguetab[2].get_rate(person.sex, person.age)
                        if len(house.people) == # would I use 0 or None?:
                            
                        if r>= rd.random():
                            
                        #relevant operations to remove person and record data
            
            

SyntaxError: invalid syntax (<ipython-input-14-d461065eb9ab>, line 28)

In [ ]:
age_of_majority = 19
eligibility = AgeTable(ages = [0,age_of_majority,100], sex1 = female, rates1 = [0,1], sex2 = male, rates2 = [0,1])
fertility = AgeTable(ages = [0,age_of_majority,age_of_majority+5,100], sex1 = female, rates1 = [0,1,0], sex2 = male, rates2 = [0,0,0])
mortality = AgeTable(ages=[0,60,100],sex1 = male, rates1 = [0,1], sex2 = female, rates2 = [0,1])
remarriage = AgeTable.NullAgeTable()
plaguetab = [housesize_5orless = AgeTable(ages = [0,20,50,100],sex1 = female, rates1 = [0.2,0.1,0.2], sex2 = male, rates2 = [0.2,0.1,0.2]), 
             housesize_5to10 = AgeTable(ages = [0,20,50,100],sex1 = female, rates1 = [0.3,0.15,0.3], sex2 = male, rates2 = [0.3,0.15,0.3]),
             housesize_morethan10 = AgeTable(ages = [0,20,50,100],sex1 = female, rates1 = [0.4,0.2,0.4], sex2 = male, rates2 = [0.4,0.2,0.4])]

In [ ]:
#The MarriageRule
marriagerule = behavior.marriage.MarriageRule(eligibility_agetable = eligibility, #defined above
                                              get_eligible = behavior.marriage.get_eligible_not_sibling_same_community,
                                              #find all single individuals in the village who aren't siblings
                                              pick_spouse = behavior.marriage.pick_spouse_random, #pick a random spouse
                                              locality = behavior.marriage.locality_patrilocality, #patrilocality
                                              remarriage_agetable = remarriage #defined above
                                             )

#The MobilityRule
def check_household_nonowner_adult_man_overcrowded(person):
    """Check that the household is overcrowded and that this person is not the owner and an adult."""
    overcrowded = behavior.mobility.check_household_overcrowded(person) #check overcrowding but also assert
    ## that there is indeed a house to check
    if overcrowded:
        if person.age >= age_of_majority and person not in person.has_house.get_owners() and person.sex == identity.male:
            #If above or at the age of majority and the person is a man who is not the owner of the house they live in
            return True
    return False

mobilityrule = behavior.mobility.MobilityRule(check_household = check_household_nonowner_adult_man_overcrowded,
                                                   who_leaves_house = behavior.mobility.who_leaves_house_family,
                                                   destination = behavior.mobility.destination_random_house_same_village)
#The InheritanceRule
#Our custom function to check sons then brothers' sons;
## note the use of lambda to make find_heirs_siblings_children into a single parameter function in order to define
## the sex as male.
find_heirs_sons_then_brothers_sons = behavior.inheritance.find_heirs_multiple_constructor(behavior.inheritance.find_heirs_sons_oldest_to_youngest,
                                                     lambda p: behavior.inheritance.find_heirs_siblings_children_oldest_to_youngest(p,sex = identity.male))


inheritancerule = behavior.inheritance.InheritanceRuleComplex(has_property = behavior.inheritance.has_property_houses,
                                                                    find_heirs = find_heirs_sons_then_brothers_sons,
                                                                    limit_heirs = behavior.inheritance.limit_heirs_not_owners,
                                                                    distribute_property = behavior.inheritance.distribute_property_to_first_heir_and_move_household,
                                                                    failure = behavior.inheritance.failed_inheritance_no_owner)

birthrule = behavior.conception.BirthRule(marriedbirth = fertility,
                                         unmarriedbirth = AgeTable.NullAgeTable(),
                                         femalesexratio = .5,
                                         maternaldeath = behavior.conception.maternal_death_zero,
                                         infanticide = behavior.conception.infanticide_none)

In [ ]:
start_pop = 100 #just a few people to start
start_area = 200 #lots of room for growth
start_age = age_of_majority - 1 #so that everyone makes the transition to marriage eligibility on time
plague = MortalityCrisis(15, 5, plaguetab)

rd.seed(1237) #Used here to standardize the example code across runs for any random parts like name generation.
terra = World()
sweetwater = Community(world = terra,
                       name = 'Sweetwater',
                       pop = start_pop,
                       area = start_area,
                       startage = start_age,
                       mortab = mortality,
                       birthtab = fertility,
                       marriagerule = marriagerule,
                       inheritancerule = inheritancerule,
                       mobilityrule = mobilityrule,
                       birthrule = birthrule)


In [ ]:
while terra.year < 43: #run for 42 years
    plague(terra)
    terra.progress()
    
print(terra.year)